<a href="https://colab.research.google.com/github/AryaFeng/pytorch/blob/main/01/pytorch_base.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

# pytorch 同时也提供了torchtext,torchVision,torchAudio的数据集

其中torchvision.datasets模块红有一个类Dataset,包含了很多的数据集，如CIFAR，COCO等。每一个TorchVision包含了两个参数，transoform和target_transform用来调整修改样本和标签。例如进行裁剪等

In [6]:
# download dataset
train_data = datasets.FashionMNIST(
    root='data',
    train=True,
    download=True,
    transform=ToTensor(),
)
# download test
test_data = datasets.FashionMNIST(
    root='data',
    train=False,
    download=True,
    transform=ToTensor(),
)

Using downloaded and verified file: data\FashionMNIST\raw\train-images-idx3-ubyte.gz
Extracting data\FashionMNIST\raw\train-images-idx3-ubyte.gz to data\FashionMNIST\raw

Using downloaded and verified file: data\FashionMNIST\raw\train-labels-idx1-ubyte.gz
Extracting data\FashionMNIST\raw\train-labels-idx1-ubyte.gz to data\FashionMNIST\raw



100%|██████████| 4422102/4422102 [00:04<00:00, 886964.45it/s] 


Extracting data\FashionMNIST\raw\t10k-images-idx3-ubyte.gz to data\FashionMNIST\raw



100%|██████████| 5148/5148 [00:00<00:00, 11491.26it/s]

Extracting data\FashionMNIST\raw\t10k-labels-idx1-ubyte.gz to data\FashionMNIST\raw



Dataset会作为参数传递给DataLoader，Dataloader支持自动批量、采样、打乱、多线程数据加载。

In [7]:
batch_size = 64

train_dataloader = DataLoader(train_data,batch_size=batch_size)
test_dataloader = DataLoader(test_data,batch_size=batch_size)

for X,y in test_dataloader:
  print(f"shapeof X [N,C,,H,W]:{X.shape}")
  print(f"shapeof y :{y.shape} {y.dtype}")
  break

shapeof X [N,C,,H,W]:torch.Size([64, 1, 28, 28])
shapeof y :torch.Size([64]) torch.int64


创建模型

In [9]:
device =( "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu" )
print(device)


# define model
class NerualNetwork(nn.Module):
  def __init__(self):
    # 构造父类
    super().__init__()
    self.flatten = nn.Flatten()  # 最后一层将图片输入展开式
    self.linear_relu_stack = nn.Sequential(
      nn.Linear(28*28,512),
      nn.ReLU(),
      nn.Linear(512,512),
      nn.ReLU(),
      nn.Linear(512,10)
    )
  def forward(self,X):
    return self.linear_relu_stack(self.flatten(X))
  
model  = NerualNetwork().to(device)
print(model)


cuda
NerualNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [10]:
loss_fn = nn.CrossEntropyLoss()
optim = torch.optim.Adam(model.parameters(),lr=1e-3)


定义训练过程，train函数

In [11]:
def train(dataloader,model,loss_fn,optim):
    size = len(dataloader)
    model.train()

    for batch,(X,y) in enumerate(dataloader):
        X, y = X.to(device),y.to(device)

        pred = model(X)
        loss = loss_fn(pred,y)

        # 反向传播
        loss.backward()  # 计算梯度
        optim.step()  # 更新梯度
        optim.zero_grad()  # 梯度清零

        if batch % 100 == 0:
            loss,current =loss.item(),batch*len(X)
            print(f"loss:{loss:>7f} [{current:>5d}/{size:>5d}]")

In [13]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()  # 保证不进行梯度计算
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [16]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optim)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss:2.296803 [    0/  938]
loss:0.557012 [ 6400/  938]
loss:0.389327 [12800/  938]
loss:0.481581 [19200/  938]
loss:0.470126 [25600/  938]
loss:0.424260 [32000/  938]
loss:0.384273 [38400/  938]
loss:0.530346 [44800/  938]
loss:0.462053 [51200/  938]
loss:0.523176 [57600/  938]
Test Error: 
 Accuracy: 84.7%, Avg loss: 0.420285 

Epoch 2
-------------------------------
loss:0.274579 [    0/  938]
loss:0.372377 [ 6400/  938]
loss:0.274577 [12800/  938]
loss:0.393451 [19200/  938]
loss:0.432758 [25600/  938]
loss:0.344004 [32000/  938]
loss:0.342018 [38400/  938]
loss:0.495209 [44800/  938]
loss:0.382036 [51200/  938]
loss:0.441435 [57600/  938]
Test Error: 
 Accuracy: 85.4%, Avg loss: 0.393177 

Epoch 3
-------------------------------
loss:0.222284 [    0/  938]
loss:0.343352 [ 6400/  938]
loss:0.219261 [12800/  938]
loss:0.350780 [19200/  938]
loss:0.405983 [25600/  938]
loss:0.324971 [32000/  938]
loss:0.294264 [38400/  938]
loss:0.466380 [44800

In [17]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


In [18]:
model = NerualNetwork().to(device)
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

In [19]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]   # testdata[0]是一个元组，第一个元素是图片，第二个元素是标签
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
